# Instagram Crawling
- chrome driver 설정
- 인스타그램 접속 후 로그인
- 해시태그별 게시글 크롤링
- 인기게시물/최근게시물 구분

In [1]:
# Visual Python: Data Analysis > Import
import numpy as np
import pandas as pd
import time
import random
import re
import os
import pickle

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

### Instagram 데이터 수집 함수 생성

In [3]:
#검색어 페이지 주소 생성 
def insta_search(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

In [4]:
# 첫 번째 게시글 선택
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, 'div._aagu')[0]
    first.click()
    time.sleep(3)

In [5]:
# 게시글 정보 가져오기 

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    try:
        content = soup.select('div._a9zs > span')[0].text # html 태그 확인
        #content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
        
    tags = re.findall(r'#[^\s#,\\]+',content)
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade')[0].findAll('span')[-1].text # html 태그 확인
    except:
        like = 0
    
    try:
        place = soup.select('div._aaql')[0].text
        # html 태그 확인
        #place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data 

In [6]:
# 다음 게시글 조회
def move_next(driver): 
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh') # html 태그 확인
    right.click()
    time.sleep(2)

### Chrome Driver 설정

In [7]:
def chromeWebdriver():
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    options = Options()
    options.add_experimental_option('detach', True)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

In [8]:
driver = chromeWebdriver()
url = 'http://www.instagram.com'
driver.get(url)
driver.implicitly_wait(3)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.68M/6.68M [00:00<00:00, 11.6MB/s]


In [9]:
#driver.quit()
#driver.close()

### Instagram 계정 설정

In [9]:
insta_id = 'seongju0705@gmail.com'
input_id = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[0] 

input_id.clear()
input_id.send_keys(insta_id)

password = 'Chosungju17&'
input_pw = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

In [10]:
# 쿠키 저장
time.sleep(10)
pickle.dump(driver.get_cookies(), open("instagram_cookies.pkl", "wb"))
cookies = pickle.load(open("instagram_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.get(url)

### Instagram 데이터 수집하기

In [11]:
# %%time 

# url = insta_search('koreagram')
# driver.get(url)
# time.sleep(20)

# select_first(driver)

# results = [ ] 

# target = 500
# for i in range(target):
#     try:
#         data = get_content(driver)
#         results.append(data)
#         move_next(driver)   
#     except:
#         time.sleep(2)
#         move_next(driver)
#     time.sleep(random.uniform(1, 4))
#     if i % 100 == 0:
#         print("{0}번 끝".format(i))
    
# print(results[:2])

# results_df = pd.DataFrame(results)
# results_df.columns = ['content', 'date', 'like', 'place', 'tags']
# results_df.to_csv('./data/koreagram_insta_crawling.csv', index=False, encoding='utf-8-sig')

In [ ]:
%%time 

url = insta_search('koreagram')
driver.get(url)
time.sleep(20)

select_first(driver)

results = [ ] 

# 이전 개수 그냥 넘기기
target1 = 11700
for i in range(target1):
    move_next(driver)
    if i % 100 == 0:
        print("{0}번 끝".format(i))

# 뒤에 이어서 크롤링
target2 = 300
for i in range(target2):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)
    time.sleep(random.uniform(1, 4))
    if i % 100 == 0:
        print("{0}번 끝".format(i))
    
print(results[:2])

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

# 원래 파일 뒤에 이어서 저장
results_df.to_csv('./data/koreagram_insta_crawling.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

0번 끝
100번 끝
200번 끝
300번 끝
400번 끝
500번 끝
600번 끝
700번 끝
800번 끝
900번 끝
1000번 끝
1100번 끝
1200번 끝
1300번 끝
1400번 끝
1500번 끝
1600번 끝
1700번 끝
1800번 끝
1900번 끝
2000번 끝
2100번 끝
2200번 끝
2300번 끝
2400번 끝
2500번 끝
2600번 끝
2700번 끝
2800번 끝
2900번 끝
3000번 끝
3100번 끝
3200번 끝
3300번 끝
3400번 끝
3500번 끝
3600번 끝
3700번 끝
3800번 끝
3900번 끝
4000번 끝
4100번 끝
4200번 끝
4300번 끝
4400번 끝
4500번 끝
4600번 끝
4700번 끝
4800번 끝
4900번 끝
5000번 끝
5100번 끝
5200번 끝
5300번 끝
5400번 끝
5500번 끝
5600번 끝
5700번 끝
5800번 끝
5900번 끝
6000번 끝
6100번 끝
6200번 끝
6300번 끝
6400번 끝
6500번 끝
6600번 끝
6700번 끝
6800번 끝
6900번 끝
7000번 끝
7100번 끝
7200번 끝
7300번 끝
7400번 끝
7500번 끝
7600번 끝
7700번 끝
7800번 끝
7900번 끝
8000번 끝
8100번 끝
8200번 끝
8300번 끝
8400번 끝
8500번 끝
8600번 끝
8700번 끝
8800번 끝
8900번 끝
9000번 끝
9100번 끝
9200번 끝
9300번 끝
9400번 끝
9500번 끝
9600번 끝
9700번 끝
9800번 끝
9900번 끝
10000번 끝


In [13]:
results_df

""


In [16]:
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

# 원래 파일 뒤에 이어서 저장
results_df.to_csv('./data/koreagram_insta_crawling.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

### 수집한 데이터 불러오기

In [3]:
# Visual Python: Data Analysis > File
result_df = pd.read_csv('./data/koreagram_insta_crawling.csv', encoding='utf-8-sig')
result_df

,content,date,like,place,tags


### 인기게시물/최근게시물 구분

In [15]:
# 인기게시물 파일 생성
pop_df = result_df.iloc[0:9]
pop_df.to_csv('./data/koreagram_pop_insta_crawling.csv', index=False, encoding='utf-8-sig')
pop_df

,content,date,like,place,tags
0,천.고.마.비의 계절 ?? ??어제와 너무나도 다른 오늘 오후언제 그랬냐는 듯이 하...,2022-09-06,"2,364",NaN,"['#daily_photography', '#fotopia', '#데일리_여행', ..."
1,#오늘의한국 #suwonsup 수원 안에 작은 중국 '월화원'괜스레 비가 오는날??...,2022-09-04,"1,187",월화원,"['#오늘의한국', '#suwonsup', '#월화원', '#효원공원', '#수원핫플']"
2,#북한산일출누구보다 휴무 알차게 보내는 사람 나야나 셔니??날씨가 너무 좋았던 지난...,2022-09-05,917,북한산 정상 백운대,"['#북한산일출누구보다', '#일출산행', '#등산스타그램', '#산스타그램', '..."
3,식장산 야경 ??태풍 전 맑은 대기와 하늘큰 태풍이 몰려오고 있다네요 큰 피해 없이...,2022-09-03,"2,152",식장산전망대,"['#daily_photography', '#fotopia', '#데일리_여행', ..."
4,..우연하게 들렸던 정방사여기서 내려다 보는 청풍호풍경이 참 좋은곳담에 가면 일몰때...,2022-09-05,"1,012",정방사,"['#첫줄', '#사진', '#취미사진', '#여행', '#일상여행', '#여행스타..."
5,어제 식장산 일몰 ?? 대구 날씨가 너무 구려서 대전으로 달려갔는데 이렇게 빛내림이...,2022-09-02,"2,351",식장산전망대,"['#daily_photography', '#fotopia', '#데일리_여행', ..."
6,Si tu o una amiga tienen planeado viajar a Cor...,2022-09-02,"5,977",NaN,"['#Corea', '#coreadelsur', '#korea', '#koreali..."
7,#여행의진수 오늘처럼 흐리고 비 오는 날 가기 좋았던 곳??수선사는 다른 절에 비해...,2022-08-31,"1,920",NaN,"['#여행의진수', '#수선사*작년']"
8,?? ‘우리 지금 떠나’ : [노들섬 노을 모음]‘우리 지금 떠나’ 시즌 두 번째예...,2022-09-04,632,노들섬,['#hongpicmagazine']


In [16]:
# 최근게시물 파일 생성
recent_df = result_df.iloc[9:]
recent_df.to_csv('./data/koreagram_recent_insta_crawling.csv', index=False, encoding='utf-8-sig')
recent_df

,content,date,like,place,tags
9,.뜨거웠던 여름작가님들과 함께한 <여수신안 :로드쇼>2022년 7월 26일~28일....,2022-09-07,0,NaN,"['#blueartfesta2022', '#GS칼텍스장도전시관', '#로드쇼2022..."
10,#엘시티레지던스 There's a fully equipped kitchen wher...,2022-09-06,0,Lct the Residence,"['#엘시티레지던스', '#5305']"
11,[J 민] [오전 7:54] #바다 #beach #여행 #koreatrip #seo...,2022-09-06,0,NaN,"['#바다', '#beach', '#여행', '#koreatrip', '#seoul..."
12,Dongdaemun Design Plaza #ddp #zahahadid #moder...,2022-09-06,3,NaN,"['#ddp', '#zahahadid', '#modernarchitecture', ..."
13,"?? Dongdaemun(동대문), Seoul",2022-09-06,0,서울,[]
...,...,...,...,...,...
995,#Chuseok (#추석) is on 10th September this year ...,2022-09-02,52,NaN,"['#Chuseok', '#추석)', '#holiday', '#명절)', '#Sou..."
996,Where in Korea is little Jimmy?! 🤨 #fun #littl...,2022-09-02,6,NaN,"['#fun', '#littlejimmy', '#koreatrip•', '#food..."
997,🎉Mes de cumpleaños = Mes de Regalos🎉[Give away...,2022-09-02,0,칠레,"['#giveaway', '#concursochile', '#bts', '#btsc..."
998,Life is a collection of moments. Cherish the b...,2022-09-02,27,서울,"['#wheninsouthkorea', '#myeongdong', '#nakwon'..."


In [29]:
#End of File